## Distortion-Based Analysis of Single Transferable Vote (Code)

This file contains code which I (Vikram Kher) have written during my time researching Single Transferable Vote (STV). The program generates a sample election scenario with c candidates and 2^(c-1) voter profiles. It then constructs and solves a linear program (via the library pulp) to find the distortion of STV in the given election scenario.

### Import Necessary Libraries

Note: pulp is a python LP solver

In [1]:
from pulp import *
import math
from string import ascii_uppercase

In [2]:
# Set up LP maximization problem
model = pulp.LpProblem("Distortion_Maximization_problem", LpMaximize)

### Generate Sample Voter Preference Lists

Note: There are many possible election scenarios which are consistent with the rules of STV; this just represents one scenario which I investigated.

In [3]:
#Choose number of candidates in election
c = 3

In [4]:
alpha = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ")
voters = []
candidates = []

v = 2**(c-1)

# Create list of voter profiles
for i in range(0, v):
    voters.append(i)


# Create list of candidates
for i in range(0,c): 
    candidates.append(alpha[i])
    
preferenceList = dict()
val = []
for i in range(0,v):
    preferenceList[voters[i]] = []
    val.append(0)
    
    
val[0] = 1/2
val[1] = 1/2
def build(preferenceList, i,j):
    for k in range(i,v):
        if(k >= 2**j):
            j = j + 1
        preferenceList[voters[k]].append(candidates[j])
        for q in range(0,c-j-1):
            preferenceList[voters[k]].append(candidates[c-q-1])
    
    build(preferenceList,i+1,j+1)
        
def rebuild(preferenceList, n):
    k = 4
    p = 2
    preferenceList[0].append(candidates[0])
    preferenceList[0].append(candidates[1])
    preferenceList[1].append(candidates[1])
    preferenceList[1].append(candidates[0])
    divide(preferenceList, val, k, p)
                

def divide(preferenceList,val, k, p):
    if k < 2**c-1:
        for i in range(0,int(k/2)):
            #Duplicate preferences
            preferenceList[i].insert(1,candidates[p])
            preferenceList[k/2+i] = preferenceList[i].copy()
            
            #Swap Candidates
            temp = preferenceList[k/2+i][0]
            preferenceList[k/2+i][0] = preferenceList[k/2+i][1]
            preferenceList[k/2+i][1] = temp
            
            #Split frequencies
            val[int(k/2)+i] = val[i]/(p+1)
            val[i]= val[i] - val[int(k/2)+i]
        
        divide(preferenceList, val, k*2,p+1)

rebuild(preferenceList, c)

In [5]:
#Prints voter profiles
print(preferenceList)

{0: ['A', 'C', 'B'], 1: ['B', 'C', 'A'], 2: ['C', 'A', 'B'], 3: ['C', 'B', 'A']}


In [6]:
#Prints frequency of voter profiles amoung voter population
print(val)

[0.33333333333333337, 0.33333333333333337, 0.16666666666666666, 0.16666666666666666]


In [7]:
# Prints labels of candidates in election
candidates

['A', 'B', 'C']

In [8]:
# Prints labels of voters in election
voters

[0, 1, 2, 3]

### Add Variables to LP

In [9]:
voter_candidate_pairs = pulp.LpVariable.dicts("pair",
                                     ((i,j) for i in voters for j in candidates),
                                     lowBound=0,
                                     cat='Continuous')

In [10]:
voter_candidate_pairs

{(0, 'A'): pair_(0,_'A'),
 (0, 'B'): pair_(0,_'B'),
 (0, 'C'): pair_(0,_'C'),
 (1, 'A'): pair_(1,_'A'),
 (1, 'B'): pair_(1,_'B'),
 (1, 'C'): pair_(1,_'C'),
 (2, 'A'): pair_(2,_'A'),
 (2, 'B'): pair_(2,_'B'),
 (2, 'C'): pair_(2,_'C'),
 (3, 'A'): pair_(3,_'A'),
 (3, 'B'): pair_(3,_'B'),
 (3, 'C'): pair_(3,_'C')}

### Add objective function

In [11]:
model += (
    pulp.lpSum([voter_candidate_pairs[(j, candidates[0])]*val[i] for i,j in enumerate(voters)])
)
model

Distortion_Maximization_problem:
MAXIMIZE
0.33333333333333337*pair_(0,_'A') + 0.33333333333333337*pair_(1,_'A') + 0.16666666666666666*pair_(2,_'A') + 0.16666666666666666*pair_(3,_'A') + 0.0
VARIABLES
pair_(0,_'A') Continuous
pair_(1,_'A') Continuous
pair_(2,_'A') Continuous
pair_(3,_'A') Continuous

### Add Constraints

#### Triangle Inequality

In [12]:
for i in voters:
    for j in candidates:
        for k in voters:
            for l in candidates:
                model += voter_candidate_pairs[(i,j)] <= voter_candidate_pairs[(k,j)] + voter_candidate_pairs[(k,l)] + voter_candidate_pairs[(i,l)]

Distortion_Maximization_problem:
MAXIMIZE
0.33333333333333337*pair_(0,_'A') + 0.33333333333333337*pair_(1,_'A') + 0.16666666666666666*pair_(2,_'A') + 0.16666666666666666*pair_(3,_'A') + 0.0
SUBJECT TO
_C1: - 2 pair_(0,_'A') <= 0

_C2: 0 pair_(0,_'A') - 2 pair_(0,_'B') <= 0

_C3: 0 pair_(0,_'A') - 2 pair_(0,_'C') <= 0

_C4: 0 pair_(0,_'A') - 2 pair_(1,_'A') <= 0

_C5: pair_(0,_'A') - pair_(0,_'B') - pair_(1,_'A') - pair_(1,_'B') <= 0

_C6: pair_(0,_'A') - pair_(0,_'C') - pair_(1,_'A') - pair_(1,_'C') <= 0

_C7: 0 pair_(0,_'A') - 2 pair_(2,_'A') <= 0

_C8: pair_(0,_'A') - pair_(0,_'B') - pair_(2,_'A') - pair_(2,_'B') <= 0

_C9: pair_(0,_'A') - pair_(0,_'C') - pair_(2,_'A') - pair_(2,_'C') <= 0

_C10: 0 pair_(0,_'A') - 2 pair_(3,_'A') <= 0

_C11: pair_(0,_'A') - pair_(0,_'B') - pair_(3,_'A') - pair_(3,_'B') <= 0

_C12: pair_(0,_'A') - pair_(0,_'C') - pair_(3,_'A') - pair_(3,_'C') <= 0

_C13: - 2 pair_(0,_'A') + 0 pair_(0,_'B') <= 0

_C14: - 2 pair_(0,_'B') <= 0

_C15: 0 pair_(0,_'B') - 2 

#### Consistency

In [13]:
for i in voters:
    for j in candidates:
        for k in candidates:
            if(preferenceList[i].index(j) < preferenceList[i].index(k)):
                model += voter_candidate_pairs[(i,j)] <= voter_candidate_pairs[(i,k)]

Distortion_Maximization_problem:
MAXIMIZE
0.33333333333333337*pair_(0,_'A') + 0.33333333333333337*pair_(1,_'A') + 0.16666666666666666*pair_(2,_'A') + 0.16666666666666666*pair_(3,_'A') + 0.0
SUBJECT TO
_C1: - 2 pair_(0,_'A') <= 0

_C2: 0 pair_(0,_'A') - 2 pair_(0,_'B') <= 0

_C3: 0 pair_(0,_'A') - 2 pair_(0,_'C') <= 0

_C4: 0 pair_(0,_'A') - 2 pair_(1,_'A') <= 0

_C5: pair_(0,_'A') - pair_(0,_'B') - pair_(1,_'A') - pair_(1,_'B') <= 0

_C6: pair_(0,_'A') - pair_(0,_'C') - pair_(1,_'A') - pair_(1,_'C') <= 0

_C7: 0 pair_(0,_'A') - 2 pair_(2,_'A') <= 0

_C8: pair_(0,_'A') - pair_(0,_'B') - pair_(2,_'A') - pair_(2,_'B') <= 0

_C9: pair_(0,_'A') - pair_(0,_'C') - pair_(2,_'A') - pair_(2,_'C') <= 0

_C10: 0 pair_(0,_'A') - 2 pair_(3,_'A') <= 0

_C11: pair_(0,_'A') - pair_(0,_'B') - pair_(3,_'A') - pair_(3,_'B') <= 0

_C12: pair_(0,_'A') - pair_(0,_'C') - pair_(3,_'A') - pair_(3,_'C') <= 0

_C13: - 2 pair_(0,_'A') + 0 pair_(0,_'B') <= 0

_C14: - 2 pair_(0,_'B') <= 0

_C15: 0 pair_(0,_'B') - 2 

#### Normalization

In [14]:
model += pulp.lpSum([voter_candidate_pairs[i, candidates[-1]]*val[voters.index(i)] for i in voters]) == 1

Distortion_Maximization_problem:
MAXIMIZE
0.33333333333333337*pair_(0,_'A') + 0.33333333333333337*pair_(1,_'A') + 0.16666666666666666*pair_(2,_'A') + 0.16666666666666666*pair_(3,_'A') + 0.0
SUBJECT TO
_C1: - 2 pair_(0,_'A') <= 0

_C2: 0 pair_(0,_'A') - 2 pair_(0,_'B') <= 0

_C3: 0 pair_(0,_'A') - 2 pair_(0,_'C') <= 0

_C4: 0 pair_(0,_'A') - 2 pair_(1,_'A') <= 0

_C5: pair_(0,_'A') - pair_(0,_'B') - pair_(1,_'A') - pair_(1,_'B') <= 0

_C6: pair_(0,_'A') - pair_(0,_'C') - pair_(1,_'A') - pair_(1,_'C') <= 0

_C7: 0 pair_(0,_'A') - 2 pair_(2,_'A') <= 0

_C8: pair_(0,_'A') - pair_(0,_'B') - pair_(2,_'A') - pair_(2,_'B') <= 0

_C9: pair_(0,_'A') - pair_(0,_'C') - pair_(2,_'A') - pair_(2,_'C') <= 0

_C10: 0 pair_(0,_'A') - 2 pair_(3,_'A') <= 0

_C11: pair_(0,_'A') - pair_(0,_'B') - pair_(3,_'A') - pair_(3,_'B') <= 0

_C12: pair_(0,_'A') - pair_(0,_'C') - pair_(3,_'A') - pair_(3,_'C') <= 0

_C13: - 2 pair_(0,_'A') + 0 pair_(0,_'B') <= 0

_C14: - 2 pair_(0,_'B') <= 0

_C15: 0 pair_(0,_'B') - 2 

#### Optimality

In [15]:
for i in candidates:
    model += pulp.lpSum([voter_candidate_pairs[j, i]*val[voters.index(j)] for j in voters]) >= 1
model

Distortion_Maximization_problem:
MAXIMIZE
0.33333333333333337*pair_(0,_'A') + 0.33333333333333337*pair_(1,_'A') + 0.16666666666666666*pair_(2,_'A') + 0.16666666666666666*pair_(3,_'A') + 0.0
SUBJECT TO
_C1: - 2 pair_(0,_'A') <= 0

_C2: 0 pair_(0,_'A') - 2 pair_(0,_'B') <= 0

_C3: 0 pair_(0,_'A') - 2 pair_(0,_'C') <= 0

_C4: 0 pair_(0,_'A') - 2 pair_(1,_'A') <= 0

_C5: pair_(0,_'A') - pair_(0,_'B') - pair_(1,_'A') - pair_(1,_'B') <= 0

_C6: pair_(0,_'A') - pair_(0,_'C') - pair_(1,_'A') - pair_(1,_'C') <= 0

_C7: 0 pair_(0,_'A') - 2 pair_(2,_'A') <= 0

_C8: pair_(0,_'A') - pair_(0,_'B') - pair_(2,_'A') - pair_(2,_'B') <= 0

_C9: pair_(0,_'A') - pair_(0,_'C') - pair_(2,_'A') - pair_(2,_'C') <= 0

_C10: 0 pair_(0,_'A') - 2 pair_(3,_'A') <= 0

_C11: pair_(0,_'A') - pair_(0,_'B') - pair_(3,_'A') - pair_(3,_'B') <= 0

_C12: pair_(0,_'A') - pair_(0,_'C') - pair_(3,_'A') - pair_(3,_'C') <= 0

_C13: - 2 pair_(0,_'A') + 0 pair_(0,_'B') <= 0

_C14: - 2 pair_(0,_'B') <= 0

_C15: 0 pair_(0,_'B') - 2 

#### Positive Distances

Already Done

### Solve Model

In [16]:
model.solve()

1

In [17]:
# Print Distortion
print(pulp.value(model.objective))

3.0


In [18]:
print("Status:", LpStatus[model.status])

Status: Optimal


In [19]:
for v in model.variables():
    if(v.name[10] == 'A' or v.name[11] == 'A'):
        print(v.name, "=", v.varValue)

pair_(0,_'A') = 2.0
pair_(1,_'A') = 4.0
pair_(2,_'A') = 2.0
pair_(3,_'A') = 4.0


### Extract dual variable values

In [20]:
for name, c in list(model.constraints.items()):
    # Print non-negative dual variables
    if(c.pi != 0):
        print(name, ":", c, "\t Dual", c.pi)

_C39 : -pair_(0,_'A') - pair_(0,_'C') + pair_(1,_'A') - pair_(1,_'C') <= 0 	 Dual -1.0014212e-13
_C45 : pair_(1,_'A') - pair_(1,_'C') - pair_(2,_'A') - pair_(2,_'C') <= 0 	 Dual 0.16666667
_C48 : pair_(1,_'A') - pair_(1,_'C') - pair_(3,_'A') - pair_(3,_'C') <= 0 	 Dual 0.16666667
_C90 : -pair_(1,_'B') - pair_(1,_'C') + pair_(2,_'B') - pair_(2,_'C') <= 0 	 Dual 0.33333333
_C111 : -pair_(0,_'A') - pair_(0,_'C') + pair_(3,_'A') - pair_(3,_'C') <= 0 	 Dual 0.33333333
_C146 : pair_(0,_'A') - pair_(0,_'C') <= 0 	 Dual 0.66666667
_C149 : pair_(1,_'B') - pair_(1,_'C') <= 0 	 Dual 0.33333333
_C151 : pair_(2,_'A') - pair_(2,_'B') <= 0 	 Dual 0.33333333
_C157 : 0.33333333333333337*pair_(0,_'C') + 0.33333333333333337*pair_(1,_'C') + 0.16666666666666666*pair_(2,_'C') + 0.16666666666666666*pair_(3,_'C') = 1.0 	 Dual 3.0
